In [1]:
!nvidia-smi

Tue Dec 31 20:16:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 48.8 MB/s eta 0:00:00


In [4]:
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import DataLoader,Dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer,
    get_linear_schedule_with_warmup
    )



In [5]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [6]:
!pip install gdown
!gdown --id 1bJylzAN7ocPTXp_ow-nLE4-hej6c68Vy #train_df.csv
!gdown --id 1hNJMOTVVKB--btCf3BLPc3frkcppw6fB #dev_df.csv

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1bJylzAN7ocPTXp_ow-nLE4-hej6c68Vy
To: /content/train_df.csv
100% 90.0M/90.0M [00:01<00:00, 87.1MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hNJMOTVVKB--btCf3BLPc3frkcppw6fB
To: /content/dev_df.csv
100% 11.2M/11.2M [00:00<00:00, 63.1MB/s]


In [7]:
squad_train_df = pd.read_csv('train_df.csv')
print(squad_train_df.shape)

squad_train_df.head()

(87599, 6)


,question,context_para,context_sent,answer_text,answer_start,answer_end
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","It is a replica of the grotto at Lourdes, Fran...",Saint Bernadette Soubirous,515,541
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",Immediately in front of the Main Building and ...,a copper statue of Christ,188,213
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",Next to the Main Building is the Basilica of t...,the Main Building,279,296
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...","Immediately behind the basilica is the Grotto,...",a Marian place of prayer and reflection,381,420
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",Atop the Main Building's gold dome is a golden...,a golden statue of the Virgin Mary,92,126


In [8]:
squad_dev_df = pd.read_csv('dev_df.csv')
print(squad_dev_df.shape)

squad_dev_df.head()

(10570, 6)


,question,context_para,context_sent,answer_text,answer_start,answer_end
0,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...,Denver Broncos,177,191
1,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...,Carolina Panthers,249,266
2,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"The game was played on February 7, 2016, at Le...","Santa Clara, California",403,426
3,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,The American Football Conference (AFC) champio...,Denver Broncos,177,191
4,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"As this was the 50th Super Bowl, the league em...",gold,488,492


In [9]:
#Using paragraph
context_name = 'context_para'
drop_context = 'context_sent'

In [10]:
df = squad_train_df.copy()
df = df.dropna()
df.rename(columns = {context_name: 'context'}, inplace=True)
df.drop(columns=[drop_context, 'answer_start', 'answer_end'], inplace=True)
test_df = df[:11877]
train_df = df[11877:]
dev_df = squad_dev_df.copy()
dev_df.rename(columns = {context_name: 'context'}, inplace=True)
dev_df.drop(columns=[drop_context, 'answer_start', 'answer_end'], inplace=True)

print(train_df.shape, 'train_df')
print(dev_df.shape, 'dev_df')
print(test_df.shape, 'test_df')

train_df.head()

(75719, 3) train_df
(10570, 3) dev_df
(11877, 3) test_df


,question,context,answer_text
11877,What is heresy mainly at odds with?,Heresy is any provocative belief or theory tha...,established beliefs or customs
11878,What is a person called is practicing heresy?,Heresy is any provocative belief or theory tha...,A heretic
11879,What religions and idea of thought is heresy c...,The term is usually used to refer to violation...,"Christianity, Judaism, Islam and Marxism"
11880,What cultures are listed as examples of discip...,"In certain historical Christian, Islamic and J...","Christian, Islamic and Jewish"
11881,What language does the term heresy find its ro...,The term heresy is from Greek αἵρεσις original...,Greek


In [11]:
SEP_TOKEN = '<sep>'
MASKING_CHANCE = 0.3 #30% chance to replace the answer with '[MASK]'

In [12]:
class QGDataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int,
        target_max_token_len: int
        ):

        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        if np.random.rand() > MASKING_CHANCE:
            answer = data_row['answer_text']
        else:
            answer = '[MASK]'

        source_encoding = tokenizer(
            '{} {} {}'.format(answer, SEP_TOKEN, data_row['context']),
            max_length= self.source_max_token_len,
            padding='max_length',
            truncation= True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        target_encoding = tokenizer(
            '{} {} {}'.format(data_row['answer_text'], SEP_TOKEN, data_row['question']),
            max_length=self.target_max_token_len,
            padding='max_length',
            truncation = True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            )

        labels = target_encoding['input_ids']
        labels[labels == 0] = -100

        return dict(
            answer_text = data_row['answer_text'],
            context = data_row['context'],
            question = data_row['question'],
            input_ids = source_encoding['input_ids'].squeeze(),
            attention_mask = source_encoding['attention_mask'].squeeze(),
            labels=labels.squeeze()
            )

In [13]:
class QGDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df: pd.DataFrame,
        val_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size,
        source_max_token_len: int,
        target_max_token_len: int
        ):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self,stage=None):
        self.train_dataset = QGDataset(self.train_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
        self.val_dataset = QGDataset(self.val_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)
        self.test_dataset = QGDataset(self.test_df, self.tokenizer, self.source_max_token_len, self.target_max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, shuffle=True, num_workers = 2)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=2)

In [14]:
MODEL_NAME = 't5-small'
SOURCE_MAX_TOKEN_LEN = 300
TARGET_MAX_TOKEN_LEN = 80

N_EPOCHS = 5
BATCH_SIZE = 16
LEARNING_RATE = 0.0001

In [15]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
print('tokenizer len before: ', len(tokenizer))
tokenizer.add_tokens(SEP_TOKEN)
print('tokenizer len after: ', len(tokenizer))
TOKENIZER_LEN = len(tokenizer)

data_module = QGDataModule(train_df, dev_df, test_df, tokenizer, BATCH_SIZE, SOURCE_MAX_TOKEN_LEN, TARGET_MAX_TOKEN_LEN)
data_module.setup()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

tokenizer len before:  32100
tokenizer len after:  32101


In [16]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=LEARNING_RATE)

In [18]:
checkpoint_callback = ModelCheckpoint(
    dirpath='/content/drive/My Drive/MCQ-CheckPoints',
    filename='mc-checkpoints',
    save_top_k=-1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

# Define the Trainer
trainer = pl.Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    accelerator='gpu',
    devices=1,
    enable_progress_bar=True,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [19]:
model = QGModel()
trainer.fit(model, data_module)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M | eval
------------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
241.971   Total estimated model params size (MB)
0         Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 14199: 'val_loss' reached 1.31711 (best 1.31035), saving model to '/content/drive/My Drive/MCQ-CheckPoints/mc-checkpoints-v2.ckpt' as top 3


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 18932: 'val_loss' reached 1.34624 (best 1.31035), saving model to '/content/drive/My Drive/MCQ-CheckPoints/mc-checkpoints-v3.ckpt' as top 4


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 23665: 'val_loss' reached 1.39330 (best 1.31035), saving model to '/content/drive/My Drive/MCQ-CheckPoints/mc-checkpoints-v4.ckpt' as top 5
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [21]:
model.eval()

QGModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32101, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32101, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_features=204

In [22]:
def generate(qgmodel: QGModel, answer: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {}'.format(answer, SEP_TOKEN, context),
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=TARGET_MAX_TOKEN_LEN,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [23]:
def show_result(generated: str, answer: str, context:str, original_question: str = ''):
    print('Generated: ', generated)
    if original_question:
        print('Original : ', original_question)

    print()
    print('Answer: ', answer)
    print('Conext: ', context)
    print('-----------------------------')

In [24]:
sample_question = test_df.iloc[42]

generated = generate(model, sample_question['answer_text'], sample_question['context'])
show_result(generated, sample_question['answer_text'], sample_question['context'], sample_question['question'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> 39.1%<sep> What percent of admissions were accepted in the early action plan?</s>
Original :  What percentage of students at Notre Dame participated in the Early Action program?

Answer:  39.1%
Conext:  Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College o

In [26]:
context = 'Oxygen is the chemical element with the symbol O and atomic number 8.'
answer = 'Oxygen'

generated = generate(model, answer, context)

show_result(generated, answer, context)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> Oxygen<sep> What is the chemical element with the symbol O?</s>

Answer:  Oxygen
Conext:  Oxygen is the chemical element with the symbol O and atomic number 8.
-----------------------------


In [27]:
context = 'Oxygen is the chemical element with the symbol O and atomic number 8.'
answer = 'Oxygen'
input_answer = '[MASK]'

generated = generate(model, input_answer, context)

show_result(generated, answer, context)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> Oxygen<sep> What is the chemical element with the symbol O?</s>

Answer:  Oxygen
Conext:  Oxygen is the chemical element with the symbol O and atomic number 8.
-----------------------------


In [28]:
for i in range(len(test_df[:10])):
    context = test_df.iloc[i]['context']
    answer = test_df.iloc[i]['answer_text']

    generated = generate(model, answer, context)

    show_result(generated, answer, context, test_df.iloc[i]['question'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> Saint Bernadette Soubirous<sep> Who was the Virgin Mary in 1858?</s>
Original :  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

Answer:  Saint Bernadette Soubirous
Conext:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
-----------------------------
Generated:  <pad> a copper statue of Christ<sep> What sta

In [30]:
for i in range(len(test_df[:10])):
    context = test_df.iloc[i]['context']
    original_answer = test_df.iloc[i]['answer_text']
    input_answer = '[MASK]'

    generated = generate(model, input_answer, context)

    show_result(generated, original_answer, context, test_df.iloc[i]['question'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated:  <pad> a golden statue of the Virgin Mary<sep> What is on the main building's gold dome?</s>
Original :  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

Answer:  Saint Bernadette Soubirous
Conext:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
-----------------------------
Generated:  <pad> a golden statue of the

In [31]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_bleu_for_qa(model, dataloader, tokenizer):

    model.eval()
    smoothing_function = SmoothingFunction().method1
    bleu_scores_per_type = {"Question": {1: [], 2: [], 3: [], 4: []},
                            "Answer": {1: [], 2: [], 3: [], 4: []}}

    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(model.device)
            attention_mask = batch['attention_mask'].to(model.device)
            labels = batch['labels'].to(model.device)

            # Generate predictions
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=TARGET_MAX_TOKEN_LEN
            )

            # Decode predictions and references
            decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True).split(SEP_TOKEN) for pred in outputs]
            decoded_refs = [
                tokenizer.decode(label[label != -100], skip_special_tokens=True).split(SEP_TOKEN) for label in labels
            ]

            # Compare predictions and references for BLEU calculation
            for pred_pair, ref_pair in zip(decoded_preds, decoded_refs):
                pred_answer, pred_question = pred_pair
                ref_answer, ref_question = ref_pair

                for gen_type, pred, ref in zip(["Answer", "Question"], [pred_answer, pred_question], [ref_answer, ref_question]):
                    pred_tokens = pred.split()
                    ref_tokens = [ref.split()]

                    for n in range(1, 5):
                        score = sentence_bleu(
                            ref_tokens,
                            pred_tokens,
                            weights=tuple([1.0 / n] * n),
                            smoothing_function=smoothing_function
                        )
                        bleu_scores_per_type[gen_type][n].append(score)

    # Compute average BLEU scores
    table_data = {
        "Type": [],
        "BLEU-1": [],
        "BLEU-2": [],
        "BLEU-3": [],
        "BLEU-4": []
    }

    for gen_type, scores_by_n in bleu_scores_per_type.items():
        table_data["Type"].append(gen_type)
        for n in range(1, 5):
            avg_score = sum(scores_by_n[n]) / len(scores_by_n[n])
            table_data[f"BLEU-{n}"].append(avg_score * 100)  # Convert to percentage format

    return pd.DataFrame(table_data)


In [32]:
test_dataset =QGDataset(test_df, tokenizer, 300, 80)
test_dataloader = DataLoader(test_dataset, batch_size=16, num_workers=2)
qa_bleu_table = evaluate_bleu_for_qa(model.model, test_dataloader, tokenizer)

# Print the table
print(qa_bleu_table)

  0%|          | 0/743 [00:00<?, ?it/s]

       Type     BLEU-1     BLEU-2     BLEU-3     BLEU-4
0  Question  27.967485  18.254354  13.088752   9.979159
1    Answer  73.357584  54.956302  40.433967  31.512250
